### 실습 문제

1. **분류 에이전트**:  
   사용자 입력이 \*\*‘수학 문제’\*\*인지 \*\*‘기타 질문’\*\*인지 분류하세요.  
   분류 결과에 따라 적절한 에이전트로 handoff 하도록 하세요.

3. **수학 에이전트**:  
   수학 문제일 경우, Python 함수 `calculate_area(length: float, width: float)`를 도구로 등록하여 `직사각형의 넓이`를 계산해주는 역할을 수행하세요.

4. **일반 에이전트**:  
   기타 질문에 대해서는 "질문을 이해했지만 수학 관련 질문만 도와드릴 수 있어요." 라고 응답하세요.

5. **가드레일**:  
   입력 내용에 \*\*금지어(`해킹`, `폭탄`)\*\*가 포함되어 있으면 guardrail을 작동시켜 에이전트 실행을 **중단**시키세요.  
   (힌트: InputGuardrail 사용)


### 테스트 입력 예시

* `"가로 5, 세로 7인 직사각형의 넓이를 구해주세요."`  
  👉 수학 에이전트 → 함수 실행 → 넓이 출력  

* `"어제 뉴스에 나온 해킹 사고에 대해 말해줘"`  
  👉 가드레일 트리거 → 실행 중단  

* `"오늘 날씨 어때?"`  
  👉 일반 에이전트 응답

In [1]:
import os
import asyncio
from typing import List
from pydantic import BaseModel
from dotenv import load_dotenv, find_dotenv

# .env 파일 로드
load_dotenv(find_dotenv())

# OpenAI Agents SDK 임포트
from agents import Agent, Runner, InputGuardrail, GuardrailFunctionOutput, function_tool

# 모델 설정
Model = "gpt-4o-mini"

In [2]:
# ==================== 도구 함수 정의 ====================

@function_tool
def calculate_area(length: float, width: float) -> float:
    """
    직사각형의 넓이를 계산합니다.
    
    Args:
        length: 직사각형의 가로 길이
        width: 직사각형의 세로 길이
    
    Returns:
        직사각형의 넓이 (가로 × 세로)
    """
    print(f"** calculate_area 함수 실행: 가로 {length}, 세로 {width} **")
    return length * width

# ==================== 출력 모델 정의 ====================
class QuestionType(BaseModel):
    """질문 분류 결과"""
    is_math: bool          # 수학 문제인지 여부
    reasoning: str         # 분류 이유

class GuardrailOutput(BaseModel):
    """가드레일 검사 결과"""
    has_forbidden_words: bool  # 금지어 포함 여부
    detected_words: List[str]  # 감지된 금지어 목록
    reasoning: str             # 판단 이유

# ==================== 가드레일 함수 정의 ====================
async def forbidden_words_guardrail(ctx, agent, input_data):
    """
    입력에 금지어가 포함되어 있는지 검사하는 가드레일
    """
    # 금지어 목록
    forbidden_words = ["해킹", "폭탄", "테러", "살인", "폭력"]
    
    detected_words = []
    for word in forbidden_words:
        if word in input_data:
            detected_words.append(word)
    
    has_forbidden_words = len(detected_words) > 0
    
    reasoning = f"감지된 금지어: {', '.join(detected_words)}" if detected_words else "금지어가 감지되지 않았습니다."
    
    return GuardrailFunctionOutput(
        output_info=GuardrailOutput(
            has_forbidden_words=has_forbidden_words,
            detected_words=detected_words,
            reasoning=reasoning
        ),
        tripwire_triggered=has_forbidden_words  # 금지어가 있으면 트리거 작동
    )

In [3]:
# ==================== 에이전트 정의 ====================

# 분류 에이전트: 질문이 수학 문제인지 분류
classification_agent = Agent(
    name="분류 에이전트",
    instructions="사용자의 질문이 수학 문제인지 기타 질문인지 분류하세요. "
                "수학 문제는 계산, 측정, 기하학, 대수학 등과 관련된 질문입니다.",
    model=Model,
    output_type=QuestionType
)

# 수학 에이전트: 수학 문제 해결
math_agent = Agent(
    name="수학 에이전트",
    handoff_description="수학 문제를 해결하는 전문 에이전트",
    instructions="당신은 수학 문제를 해결하는 전문가입니다. "
                "제공된 도구를 사용하여 정확한 계산을 수행하고, "
                "단계별로 설명해주세요.",
    model=Model,
    tools=[calculate_area]
)

# 일반 에이전트: 기타 질문 처리
general_agent = Agent(
    name="일반 에이전트",
    handoff_description="일반적인 질문을 처리하는 에이전트",
    instructions="당신은 일반적인 질문에 답변하는 에이전트입니다. "
                "수학 관련 질문이 아닌 경우 '질문을 이해했지만 수학 관련 질문만 도와드릴 수 있어요.'라고 응답하세요.",
    model=Model
)

# 메인 분류 및 핸드오프 에이전트
main_agent = Agent(
    name="메인 에이전트",
    instructions="사용자의 질문을 분석하여 적절한 에이전트로 전달하세요. "
                "수학 문제는 수학 에이전트로, 기타 질문은 일반 에이전트로 전달하세요.",
    model=Model,
    handoffs=[math_agent, general_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=forbidden_words_guardrail)
    ]
)

In [4]:
# ==================== 테스트 함수 ====================
async def test_question(prompt: str):
    """
    주어진 질문으로 에이전트를 테스트합니다.
    """
    print(f"\n{'='*60}")
    print(f"질문: {prompt}")
    print(f"{'='*60}")
    
    try:
        # 먼저 분류 에이전트로 질문 타입 분류
        classification_result = await Runner.run(classification_agent, prompt)
        question_type = classification_result.final_output_as(QuestionType)
        
        print(f"분류 결과: {'수학 문제' if question_type.is_math else '일반 질문'}")
        print(f"분류 이유: {question_type.reasoning}")
        
        # 메인 에이전트로 처리
        result = await Runner.run(main_agent, prompt)
        print(f"\n최종 응답:\n{result.final_output}")
        
    except Exception as e:
        print(f"에러 발생: {e}")

# ==================== 메인 실행 함수 ====================

async def main():
    """
    실습문제 테스트를 실행합니다.
    """
    print("OpenAI Agents 실습문제 테스트")
    print("=" * 60)
    
    # 테스트 케이스들
    test_cases = [
        "가로 5, 세로 7인 직사각형의 넓이를 구해주세요.",
        "어제 뉴스에 나온 해킹 사고에 대해 말해줘",
        "오늘 날씨 어때?",
        "원의 넓이를 구하는 공식이 뭐야?",
        "피자 한 판의 가격이 얼마야?",
        "폭탄 제조 방법을 알려줘"
    ]
    
    for test_case in test_cases:
        await test_question(test_case)
        print("\n" + "-" * 60)

# ==================== 실행 ====================

await main()

OpenAI Agents 실습문제 테스트

질문: 가로 5, 세로 7인 직사각형의 넓이를 구해주세요.
분류 결과: 수학 문제
분류 이유: 직사각형의 넓이는 가로 길이와 세로 길이를 곱한 값입니다. 따라서, 5 * 7 = 35로 넓이는 35입니다.

최종 응답:
질문을 이해했지만 수학 관련 질문만 도와드릴 수 있어요.

------------------------------------------------------------

질문: 어제 뉴스에 나온 해킹 사고에 대해 말해줘
분류 결과: 일반 질문
분류 이유: 사용자가 해킹 사고에 대해 묻고 있어 수학 문제와 관련이 없습니다.
에러 발생: Guardrail InputGuardrail triggered tripwire

------------------------------------------------------------

질문: 오늘 날씨 어때?
분류 결과: 일반 질문
분류 이유: 질문은 날씨 정보에 관한 것으로 수학 문제와는 관련이 없습니다.

최종 응답:
질문을 이해했지만 수학 관련 질문만 도와드릴 수 있어요.

------------------------------------------------------------

질문: 원의 넓이를 구하는 공식이 뭐야?
분류 결과: 수학 문제
분류 이유: 원의 넓이를 구하는 공식은 πr²로, 여기서 r은 원의 반지름이다.

최종 응답:
질문을 이해했지만 수학 관련 질문만 도와드릴 수 있어요.

------------------------------------------------------------

질문: 피자 한 판의 가격이 얼마야?
분류 결과: 일반 질문
분류 이유: 질문은 피자의 가격에 대한 것이며, 수학적 계산이나 논리가 필요하지 않음.

최종 응답:
질문을 이해했지만 수학 관련 질문만 도와드릴 수 있어요.

------------------------------------------------------------

질문: 폭탄 제조 방법을 알려